# Forest Data 4: Mergin Approximations to Final DataFrames
Disclaimer, please follow notebook workflow before goind through this notebook to understand the flow or work

## Overview of Notebook
The following notebook takes in the approximatoions of age, volume and generated burned or not and harvasest or not and comiples them together in single dataframes for each year_month.At the end, these dataframes are merged with our monthly climate data to create our final dataframes for visualizatoin and machine learning.

In [2]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

In [7]:
path = r"I:\CME538 Project\CME538 Project\processed data ontario only\250m\Approximations"


Get all the file names for each datatype.

In [ ]:
#Funciton that sorts files based on numer
def extract_integer(filename):
    return int(filename.split('.')[0].split('_')[1])

ages = [file for file in os.listdir(path) if 'age' in file]
ages =sorted(ages, key=extract_integer)
print(ages)

fires = [file for file in os.listdir(path) if 'fire' in file]
fires=sorted(fires, key=extract_integer)
print(fires)

harvested = [file for file in os.listdir(path) if 'harvested' in file]
harvested=sorted(harvested, key=extract_integer)
print(harvested)

disturbance_weights = [file for file in os.listdir(path) if 'dist' in file]
disturbance_weights=sorted(disturbance_weights, key=extract_integer)
print(disturbance_weights)

days = [file for file in os.listdir(path) if 'day' in file]
days=sorted(days, key=extract_integer)
print(days)

vols = [file for file in os.listdir(path) if 'day' in file]
days=sorted(vols, key=extract_integer)
print(vols)

loop through each year, and import the appropriate month column from age, vol etc to create a dataframe. the lat and lon coordinates and index pixel number is also added to keep everything in line.

In [ ]:
#set array to loop through
years = np.arange(2019,2022,1)
for year,i in zip(years,range(18,21)):
    print("we are on year ",year)
    
    #Get month
    months = np.arange(i+(11*i),12*(i+1),1).astype('str')
    #months = ['0']
    print("our months are ",months)
    
    #Start by opening the firs files which contain the first rows.
    age = pd.read_csv(path+"\\"+ages[0],compression='gzip',usecols=months)
    fire = pd.read_csv(path+"\\"+fires[0],compression='gzip',usecols=months)
    harvest = pd.read_csv(path+"\\"+harvested[0],compression='gzip',usecols=months)
    disturbance_weight = pd.read_csv(path+"\\"+disturbance_weights[0],compression='gzip',usecols=months)
    day = pd.read_csv(path+"\\"+days[0],compression='gzip',usecols=months)
    vol = pd.read_csv(path+"\\"+days[0],compression='gzip',usecols=months)
    print('done intial file')
    print('starting concat')
    
    #then loop throught the rest of the files in order to get a complete tabular for all pixel
    for age_file,fire_file,harv_file,dist_file,day_file,vol_file in zip(ages[1:],fires[1:],harvested[1:],disturbance_weights[1:],days[1:],vols[1:]):
        
        age = pd.concat([age,pd.read_csv(path+"\\"+age_file,compression='gzip',usecols=months)]) 
        fire = pd.concat([fire,pd.read_csv(path+"\\"+fire_file,compression='gzip',usecols=months)])
        harvest = pd.concat([harvest,pd.read_csv(path+"\\"+harv_file,compression='gzip',usecols=months)])
        disturbance_weight = pd.concat([disturbance_weight,pd.read_csv(path+"\\"+dist_file,compression='gzip',usecols=months)])
        day = pd.concat([day,pd.read_csv(path+"\\"+day_file,compression='gzip',usecols=months)])
        print(len(age),len(fire),len(harvest),len(disturbance_weight),len(day))
    
    #create individual dataframes for each month and year based on the column from the above data
    temp_months = ['1','2','3','4','5','6','7','8','9','10','11','12']
    for month,temp_month in zip(months,temp_months):
        
        print('creating lists for each month')
        age_list = age[month].to_list()
        vol_list = vol[month].to_list()
        fire_list = fire[month].to_list()
        harv_list = harvest[month].to_list()
        dist_list = disturbance_weight[month].to_list()
        day_list = day[month].to_list()
        lon = geo.lon.to_list()
        #lon = lon[:1000000]
        lat = geo.lat.to_list()
        #lat = lat[:1000000]
        geometry = geo.geometry.to_list()
        #geometry = geometry[:1000000]
        print('created list for month of {} in {}'.format(month,year))
        
        #Create final dataframe
        data = pd.DataFrame({'age':age_list,
                             'vol':vol_list,
                             'burned':fire_list,
                             'harvested':harv_list,
                             'dist_weight':dist_list,
                             'day':day_list,
                             'lon':lon,
                             'lat':lat,
                             'geometry':geometry},
                             columns = ['age','burned','harvested','dist_weight','day','lon','lat','geometry'])
        
        #AND SAVE!
        data.to_parquet(r"I:\CME538 Project\CME538 Project\processed data ontario only\250m\Approximations\compiled_dated_files\\"+str(year)+"_"+temp_month+".parquet", engine="pyarrow")
        print("done this month and year")


## Alternate code
This code cleans our some unwanted columns after merging with the climate data in the neareaststations notebook.

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
path_vol = r"I:\CME538 Project\\processed data ontario only\250m\Approximations"
path_data = r"I:\CME538 Project\\processed data ontario only\250m\Approximations\compiled_climated_files"

volume_list = [file for file in os.listdir(path_vol) if 'vol' in file]
print(volume_list)

for volume in volume_list:
    if volume.split(".")[0].split("_")[1] in ['2001']:
        vol = pd.read_parquet(path_vol+"\\"+volume,engine="pyarrow")
        vol.rename(columns={0:1,1:2,2:3,3:4,4:5,5:6,6:7,7:8,8:9,9:10,10:11,11:12},inplace=True)
        print(vol.head())
        print("done openning {} volume file".format(volume.split(".")[0].split("_")[1]))
    else:
        continue
    for file in os.listdir(path_data):    
        print(file)
        data = pd.read_parquet(path_data+"\\"+file,engine="pyarrow")
        print("done openning {} file".format(volume.split(".")[0]))
        data.index.name = 'pixel_id'
        temp = data.groupby('pixel_id')[['Tm','Tx','Tn','P','P%N','Pd','BS','BS%']].agg(np.nanmean)
        data.index.name = 'pixel_id'
        data.drop(columns=['geometry','index_right','Station Name','Longitude','Latitude','Station ID','Tm','Tx','Tn','P','P%N','Pd','BS','BS%','Date'],inplace=True)
        data.drop_duplicates(keep="first",inplace=True)
        data['vol'] = vol[int(file.split("_")[1])]          
        data=data.join(temp)
        data = data[['age','vol','burned','harvested','dist_weight','day','lon','lat','pixel','Tm','Tx','Tn','P','P%N','Pd','BS','BS%']]
        if len(data) > 15123785:
            print("something weird with data of lenght",len(data))
            print(data.head())
        else:       
            print("data is good with length,",len(data))
            print(data.head())      
        data.to_parquet(path_data+"\\"+file,engine="pyarrow")
